In [1]:
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable

import os

import matplotlib.pyplot as plt
import numpy as np

C:\Users\Administrator\anaconda3\envs\AE\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_transform = transforms.Compose([
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.ToTensor()
])
#设置数据集
class ImageDataset(Dataset):
    
        def __init__(self,x_path,y_path,tfm = test_transform):
            super(ImageDataset).__init__()
            self.x_path = x_path
            self.y_path = y_path
            self.files_x = sorted([os.path.join(x_path,x) for x in os.listdir(x_path) if x.endswith(".jpg")])
            self.files_y = sorted([os.path.join(y_path,y) for y in os.listdir(y_path) if y.endswith(".jpg")])
            self.transforms = tfm
    
        def __len__(self):
            return len(self.files_x)

        def __getitem__(self,idx):
            fname_x = self.files_x[idx]
            fname_y = self.files_y[idx]
            im_x = Image.open(fname_x)
            im_y = Image.open(fname_y)
            im_x = self.transforms(im_x)
            im_y = self.transforms(im_y)
            return im_x, im_y

In [4]:
batch_size = 1
noise_dir = "NoisedImages"
origin_dir = "OriginalImages"
train_set = ImageDataset(noise_dir,origin_dir,train_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

In [3]:
#定义自编码器模型
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.batchnorm import BatchNorm2d
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder,self).__init__()
        self.Encoder = nn.Sequential(
            nn.Conv2d(3,64,3,1,1),
            nn.ReLU(),
            
            nn.Conv2d(64,64,3,1,1),
            nn.ReLU(),
        )
        
        self.Decoder = nn.Sequential(
          

            nn.ConvTranspose2d(64,64,3,1,1),
            nn.ReLU(),

            nn.ConvTranspose2d(64,3,3,1,1),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        encoder = self.Encoder(x)
        decoder = self.Decoder(encoder)
        return encoder, decoder

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Selected device: {device}')
model = AutoEncoder().to(device)

Selected device: cuda


In [11]:
#损失函数 配置参数
torch.manual_seed(1)

n_epochs=100
LR=0.001
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-05)

In [12]:
#定义训练过程
from tqdm.auto import tqdm
from PIL import Image

for epoch in range(n_epochs):
    
        # ------------Training------------
        # Make sure the model is in train mode before training
        model.train()
        
        train_loss = []
        
        for image_noise, image_clean in tqdm(train_loader):
                
                image_noise = image_noise.to(device)
                image_clean = image_clean.to(device)
                
                _, autoencoder_data = model(image_noise)
                resize = transforms.Resize(list(autoencoder_data.shape)[2:])
                print(list(autoencoder_data.shape))
                image_clean = resize(image_clean)
                
                loss = loss_fn(autoencoder_data,image_clean)
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                
                # Update the parameters with computed gradients.
                optimizer.step()
                
                train_loss.append(loss.item())
        
        train_loss = sum(train_loss)/len(train_loss)
        
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")

C:\Users\Administrator\anaconda3\envs\AE\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

[1, 3, 350, 500]


 50%|██████████████████████████████████████████                                          | 2/4 [00:07<00:06,  3.02s/it]

[1, 3, 593, 899]
[1, 3, 389, 740]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.86s/it]


...

[1, 3, 389, 740]
[1, 3, 350, 500]


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:00<00:00, 11.71it/s]

[1, 3, 533, 800]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 10.14it/s]

[ Train | 100/100 ] loss = 0.00307


In [1]:
#储存经自编码器降噪后的照片
model.eval()
import cv2
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
index = 1

for image_noise, image_clean in train_loader:
    image_noise = image_noise.to(device)
    _, output = model(image_noise)
    im_denoise = np.transpose(output.cpu().data.numpy(),(0,2,3,1))
    im_denoise = cv2.cvtColor(im_denoise[0,...],cv2.COLOR_RGB2BGR)

    cv2.imwrite(f"{index}.jpg", np.clip(im_denoise*255,0,255))

    index+=1

NameError: name 'model' is not defined